In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from download_tools.plugins.mouselab_mdp import preprocess_mouselab_data, add_click_count_columns
from download_tools.plugins.survey_html_form import process_html_demographics
from download_tools.plugins.survey_multi_choice import score_mouselab_questionnaires, get_mouselab_quiz_name, get_quiz_passer_ids, score_row
from download_tools.plugins.survey_text import preprocess_survey_text, get_old_demographics
from download_tools.plugins.utils import get_demo_string

from datetime import datetime
import json

In [2]:
# Parameters
database_key = "NEW"
bonus_function = "lambda row: round((row[\"score\"] + 50) * .002, 2)"
simulated = False
sessions = ["methods_main", "methods_main_extension"]
experiment_setting = "high_increasing"
COST = {"irlexp10_1": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
DEPTH = {"irlexp10_1": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
html_survey_names = {}
old_experiment = False
manual_age_mapping = {}
experiment_specific_mapping = {}
trials_per_block = {"test": 20, "training": 20}
ranges_to_extract = None
mouselab_column_identifier = "name"
mouselab_mapping = {"0.0-1.*-1.*": "mouselab-quiz-pre", "0.0-3.*-3*": "mouselab-quiz-post", "0.0-3.*-2*": "mouselab-quiz-post"}
max_attempts = {"mouselab-quiz-pre": 4, "mouselab-quiz-post": 1}
passing_score = {"mouselab-quiz-pre": 5, "mouselab-quiz-post": 0}
mouselab_quiz_solutions = {"mouselab-quiz-pre": {"Q0": "$-48 to $48", "Q1": "No, the cost is always $1.00.", "Q2": "There is no limit", "Q3": "The better I perform the higher my bonus will be.", "Q4": "No, the amount of cash at each node of the web may be different each time."}, "mouselab-quiz-post": {"Q0": "$-4 to $4", "Q1": "$-8 to $8", "Q2": "$-48 to $48", "Q3": "No, the cost is always $1.00.", "Q4": {"Very unmotivated": -2, "Slightly unmotivated": -1, "Neither motivated nor unmotivated": 0, "Slightly motivated": 1, "Very motivated": 2, "": 0}}}
ground_truth_file = "312_2_4_24"
node_classification = {"early": [1, 5, 9], "middle": [2, 6, 10], "late": [3, 4, 7, 8, 11, 12], "clicks": [1, 5, 9, 2, 6, 10, 3, 4, 7, 8, 11, 12]}
structure = "312_2_4_24"
analysis_run = "methods_main"
data_path = "/home/vfelso/github/planning-depth-differences/data"


In [3]:
# paths to use
inputs_path = Path(data_path).joinpath('inputs')
raw_data_path = Path(data_path).joinpath('raw')
processed_data_path = Path(data_path).joinpath(f'processed/{analysis_run}')
processed_data_path.mkdir(parents=True, exist_ok=True)

In [4]:
with open(inputs_path.joinpath(f"exp_inputs/rewards/{ground_truth_file}.json")) as json_file:
    ground_truths = json.load(json_file)

In [5]:
# load data
full_data = {}

# read in sessions
for run in sessions:
    for file_path in raw_data_path.glob(f"{run}/*.csv"):
        # don't want to save identifiable bonuses
        # file, information is already in data
        if "bonuses" not in str(file_path):
            file_name = file_path.stem
            curr_data_frame = pd.read_csv(file_path)
            curr_data_frame["run"] = run
            if file_name not in full_data:
                full_data[file_name] = [curr_data_frame]
            else:
                full_data[file_name].append(curr_data_frame)

full_data = {k: pd.concat(v) for k,v in full_data.items()}

In [6]:
individual_variables = full_data["general_info"].merge(full_data["question_data"], on=["pid","run"])

time_format = '%Y-%m-%d %H:%M:%S.%f'
finished_df = individual_variables[individual_variables["endhit"].apply(lambda endhit: isinstance(endhit, str))].reset_index()
finished_df["time_diff"] = finished_df.apply(lambda row: ((datetime.strptime(row["endhit"], time_format) - datetime.strptime(row["beginhit"], time_format)).seconds % 3600 )/ 60.0, axis=1)
individual_variables = individual_variables.merge(finished_df[["time_diff", "pid", "run"]], how="left", on=["pid", "run"])

# check saved cost makes sense
if (COST is not None) and (DEPTH is not None):
    if isinstance(COST, dict):
        assert(np.all(individual_variables.apply(lambda row: row["COST"] == COST[row["codeversion"]][int(row["cond"])],axis=1)))
    else:
        unique_costs = np.unique(individual_variables["COST"])
        assert(len(unique_costs) == 1)
        assert(unique_costs[0] == COST)
    if DEPTH:
        if isinstance(DEPTH, dict):
            assert(np.all(individual_variables.apply(lambda row: row["DEPTH"] == DEPTH[row["codeversion"]][int(row["cond"])],axis=1)))
        else:
            unique_costs = np.unique(individual_variables["DEPTH"])
            assert(len(unique_costs) == 1)
            assert(unique_costs[0] == DEPTH)
    print(finished_df.groupby(["DEPTH", "COST"]).mean())
else:
    print(finished_df.groupby(["cond"]).mean())

                 index       cond  counterbalance  bonus  status          pid  \
DEPTH COST                                                                      
0     0     102.888889  11.355556             0.0    0.0     3.0  2252.140741   

            MIN_TIME  inspectCost  bonusRate  branching  final_bonus  \
DEPTH COST                                                             
0     0          7.0          1.0      0.002      312.0     2.692984   

            displayed_bonus  time_diff  
DEPTH COST                              
0     0            2.687339  15.115309  


In [7]:
survey_texts = preprocess_survey_text(full_data["survey-text"])

if old_experiment:
    demographics, demo_text = get_old_demographics(survey_texts, experiment_specific_gender=experiment_specific_mapping, manual_age_mapping=manual_age_mapping)
else:
    html_survey = full_data["survey-html-form"]
    if len(html_survey_names) > 0:
        if "name" not in html_survey:
            html_survey["name"] = np.nan

        html_survey["name"] = html_survey.apply(
            lambda row: get_mouselab_quiz_name(row, html_survey_names[row["run"]]) if not isinstance(row["name"], str) else row["name"],
            axis=1)
        demographics, demo_text = process_html_demographics(html_survey[html_survey["name"] == "demographics"])
        full_data["survey-multi-choice"] = pd.concat([full_data["survey-multi-choice"], html_survey[html_survey["name"] != "demographics"]])
    else:
        demographics, demo_text = process_html_demographics(full_data["survey-html-form"])
    
print(demo_text)

56 females, 74 males, 1 others; median age 42, age range 20-75


In [8]:
mouselab_datas = preprocess_mouselab_data(full_data["mouselab-mdp"],trials_per_block,ground_truths)

mouselab_datas = mouselab_datas.merge(individual_variables, how="left", on=["pid", "run"])

# path may contain a bunch of 0s at the start due to miscoding
mouselab_datas["path"] = mouselab_datas["path"].apply(lambda path : eval(path)[-3:])

node_classification = {key : [str(node) for node in val] for key, val in node_classification.items()}
mouselab_datas = add_click_count_columns(mouselab_datas, node_classification)

In [9]:
#TODO would be nice to refactor this out and import it
def expand_range_dictionary(input_dictionary):
    trial_to_block = {}
    for block, trial_range in input_dictionary.items():
        if isinstance(trial_range, str):
            for trial_index in eval(trial_range):
                trial_to_block[trial_index] = block
        else:
            trial_to_block[block] = expand_range_dictionary(trial_range)
    return trial_to_block

if ranges_to_extract:
    trial_to_block = expand_range_dictionary(ranges_to_extract)

    mouselab_datas["block"] = mouselab_datas.apply(lambda row: trial_to_block[row["run"]][row["trial_index"]] if row["run"] in trial_to_block else trial_to_block[row["trial_index"]], axis=1)

In [10]:
questionnaires = full_data["survey-multi-choice"]

if "name" not in questionnaires:
    questionnaires["name"] = np.nan

questionnaires["name"] = questionnaires.apply(
    lambda row: get_mouselab_quiz_name(row, mouselab_mapping) if not isinstance(row["name"], str) else row["name"],
    axis=1)

mouselab_questionnaires = questionnaires[questionnaires["name"].isin(mouselab_mapping.values())].reset_index()
questionnaires = questionnaires[~questionnaires["name"].isin(mouselab_mapping.values())].reset_index()

mouselab_questionnaires["name"] = mouselab_questionnaires.apply(lambda row: get_mouselab_quiz_name(row, mouselab_mapping) if not isinstance(row["name"], str) else row["name"],axis=1)
mouselab_questionnaires = score_mouselab_questionnaires(mouselab_questionnaires, mouselab_quiz_solutions, mouselab_column_identifier)
mouselab_quiz = mouselab_questionnaires.drop_duplicates(["pid","question_id"], keep="last")

pivoted_mouselab_quiz = mouselab_quiz.pivot_table(values="score", index=["pid","run"], columns="question_id")

In [11]:
if len(questionnaires)>0:
    raise NotImplementedError

In [12]:
quiz_passers = get_quiz_passer_ids(mouselab_questionnaires, max_attempts=max_attempts, passing_score=passing_score, identifying_columns = ["pid", "run"])
passed_all_quizzes = list(set.intersection(*map(set,quiz_passers.values())))
print(len(passed_all_quizzes))
pivoted_mouselab_quiz["passed_quizzes"] = 0
pivoted_mouselab_quiz.loc[passed_all_quizzes, "passed_quizzes"] = 1

/home/vfelso/github/planning-depth-differences/venv/lib/python3.8/site-packages/download_tools/plugins/survey_multi_choice.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data["attempt_num"] = survey_data["internal_node_id"].apply(


122


In [13]:
quiz_and_demo = pivoted_mouselab_quiz.join(demographics)
quiz_and_demo = quiz_and_demo.merge(individual_variables, how="left", on=["pid", "run"])

quiz_and_demo_subselection = quiz_and_demo[quiz_and_demo.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)]

gender_values, gender_counts = np.unique(quiz_and_demo_subselection["gender"].values, return_counts = True)
print(get_demo_string(list(map(int, quiz_and_demo_subselection["age"].values)), gender_counts, gender_values))


if len(questionnaires)>0:
    scored_questionnaire_df.to_csv(processed_data_path.joinpath("questionnaires.csv"))
quiz_and_demo[quiz_and_demo.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("quiz-and-demo.csv"))
mouselab_datas[mouselab_datas.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("mouselab-mdp.csv"))
survey_texts[survey_texts.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("survey-text.csv"))
individual_variables[individual_variables.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].to_csv(processed_data_path.joinpath("individual-variables.csv"))

52 females, 69 males, 1 others; median age 42, age range 20-75


In [ ]:
median_info = finished_df[finished_df.apply(lambda row: (row["pid"], row["run"]) in \
                                 passed_all_quizzes, axis=1)].median()
print(f"median time: {median_info['time_diff']:.2f}, median bonus: {median_info['final_bonus']:.2f}")